In [1]:
import pandas as pd
import os
import numpy as np
from copy import deepcopy

In [2]:
st = pd.HDFStore(os.path.expanduser("~/all.h5"), mode='r')

In [3]:
metadata_df = pd.read_csv("metadata/metadata.csv",index_col=0)

In [4]:
feeds = {'use':'aggregate',
        'air1':'hvac',
         'clotheswasher1':'wm',
         'dishwasher1':'dw',
         'microwave1':'mw',
         'refrigerator1':'fridge',
         'oven1':'oven',
        'car1':'ec',
         'waterheater1':'wh',
         'drye1':'dr'
        }

In [5]:
feed_r = {v:k for k, v in feeds.iteritems()}

In [6]:
cities = ['Austin','Houston','San Diego','Boulder']

In [7]:
years = {
    2014:{'start':'1-1-2014', 'end':'12-31-2014', 'start_month':1, 'end_month':12, 'months':1},
    2015:{'start':'1-1-2015', 'end':'12-31-2015', 'start_month':1, 'end_month':12, 'months':1},
    2016:{'start':'1-1-2016', 'end':'12-31-2016', 'start_month':1, 'end_month':12, 'months':1}
    }

In [8]:
import pickle
out_overall = pickle.load(open('metadata/all_regions_years_cleaned.pkl', 'r'))

In [9]:
a = out_overall[2014]['Austin'].index

In [10]:
import sys
sys.path.append('../aaai/')
from create_matrix import *
df, dfc = create_matrix_single_region('Austin', 2014)

In [11]:
home, appliance, month = 2378,'hvac',6
df.ix[home]['%s_%d' %(appliance, month)]

nan

In [12]:
df.shape

(533, 128)

In [13]:
df.size

68224

In [14]:
df.notnull().sum().sum()

27502

In [15]:
df[['dw_%d' %month for month in range(1, 13)]].describe()

,dw_1,dw_2,dw_3,dw_4,dw_5,dw_6,dw_7,dw_8,dw_9,dw_10,dw_11,dw_12
count,205.000000,222.000000,252.000000,259.000000,265.000000,274.000000,285.000000,291.000000,297.000000,303.000000,299.000000,298.000000
mean,9.708619,8.434810,8.728374,8.144696,8.160846,7.714047,7.565975,7.764865,8.070098,8.728551,9.352359,9.855035
std,9.742111,8.173240,8.310532,8.073685,8.135321,7.426026,7.304776,7.692711,8.112861,8.748531,9.043597,9.821733
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.411550,1.665383,1.383867,0.693025,0.771567,1.167817,1.244183,1.073133,1.249450,1.590183,1.839717,1.631142
50%,7.270467,6.552008,7.114942,5.709383,6.221283,5.898283,5.984983,5.888183,5.959017,6.269166,7.082933,7.243392
75%,14.768400,12.659258,12.734054,13.042058,12.649517,12.357571,11.418867,11.982325,11.814983,13.113583,14.645150,15.934863
max,48.084332,43.614601,36.097450,36.975132,37.158764,38.095615,35.783581,35.976170,38.892448,44.840683,42.959618,47.639168


In [15]:
df[['oven_%d' %month for month in range(1, 2)]]

,oven_1
22,NaN
26,20.389349
48,NaN
59,14.296000
68,NaN
77,NaN
86,NaN
93,5.864000
94,NaN
101,NaN


In [16]:
%matplotlib tk

In [17]:
import matplotlib.pyplot as plt
from matplotlib.widgets import Button, CheckButtons

fig, ax = plt.subplots()
fig.subplots_adjust(bottom=0.2)
fig.subplots_adjust(left=0.2)

home_0 = st['/%d' %df.index.values[0]]

rax = plt.axes([0.02, 0.4, 0.13, 0.2], aspect='equal')

appliances = ['air1','use']
year='2014'
labels = tuple(appliances)
states = tuple([True]*len(labels))
check = CheckButtons(rax, labels, states)


home_0[appliances][year].plot(ax=ax, title="Sample number: 0").legend(loc=2)
lines = ax.get_lines()

class Index:
    store = st
    data_min = 0
    data_max = len(df)-1
    selected = 0
    home_num = df.index.values[selected]
    st, la = states, labels
    states_dict = dict(zip(la, st))
    def selected_column(self, label):
        self.states_dict[label] = not self.states_dict[label]
        self.plot()
    
    def plot(self):
        ax.cla()
        home_num = df.index.values[self.selected]
        home_df = st['/%d' %home_num][year]
        if 'air1' in home_df.columns:
            home_df[appliances].plot(ax=ax, title="Home: %d" %home_num).legend(loc=2)
        else:
            home_df[['use']].plot(ax=ax, title="Home: %d" %home_num).legend(loc=2)
       
        lines = ax.get_lines()
        for i ,(l, s) in enumerate(self.states_dict.items()):
            lines[i].set_visible(s)
        plt.legend(loc=1)
        
        
    def next(self, event):
        if self.selected >=self.data_max:
            self.selected = self.data_max
            ax.set_title('Last sample reached. Cannot go forwards')
        else:
            self.selected += 1
            self.plot()
            

    def prev(self, event):
        if self.selected <=self.data_min:
            self.selected = 0
            ax.set_title('First sample reached. Cannot go backwards')
        else:
            self.selected -= 1
            self.plot()
        

callback = Index()
axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axnext = plt.axes([0.81, 0.05, 0.1, 0.075])

bnext = Button(axnext, '>')
bnext.on_clicked(callback.next)

bprev = Button(axprev, '<')
bprev.on_clicked(callback.prev)

check.on_clicked(callback.selected_column);

In [24]:
for appliance in APPLIANCES_ORDER:
    for month in range(1, 13):
        

In [43]:
appliance, month = 'hvac', 1
d = df['%s_%d' %(appliance, month)]
e = (d-d.median()).abs().sort_values()[::-1]
e.dropna().head(10)

5357    865.795631
59      723.118385
1700    643.282325
1782    641.330360
5673    626.841468
8419    622.612281
7875    517.472389
5109    485.789711
3192    449.892463
8669    401.957191
Name: hvac_1, dtype: float64

In [46]:
st['/1700']['2014'][feed_r[appliance]].plot()

In [ ]:
to_remove = 
 {
     'fridge':
     {
         1: [5371, # never turns off,
             3652, # never turns off,
             9019, # never turns off,
             2864, # never turns off,
            ]
     },
     'hvac':
     {
         1: [5357, # high HVAC energy through the year
            59,
            ]
     }
             
             
             
         
     }
 }

In [12]:
def pl(home, appliances=['aggregate','hvac']):
    for ap in appliances:
        print df.ix[home][['%s_%d' %(ap, month) for month in range(1, 13)]]

In [19]:
pd.value_counts(st['/1185']['2014'].resample('1d',how='mean')['use'].dropna().index.month)

/Users/nipunbatra/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  """Entry point for launching an IPython kernel.


12    31
8     31
7     31
5     31
3     31
1     31
9     30
6     30
4     30
2     28
10     8
11     7
dtype: int64

In [13]:
pl(1185)

aggregate_1      849.482971
aggregate_2      742.265991
aggregate_3      766.720459
aggregate_4      862.627563
aggregate_5     1063.473633
aggregate_6     1551.017456
aggregate_7     1952.096802
aggregate_8     2054.313477
aggregate_9     1567.942505
aggregate_10     379.430298
aggregate_11     189.506805
aggregate_12     891.225159
Name: 1185, dtype: float64
hvac_1       3.457550
hvac_2       1.322767
hvac_3       9.706167
hvac_4     140.863190
hvac_5     281.126373
hvac_6     658.252075
hvac_7     848.315063
hvac_8     988.086243
hvac_9     626.911316
hvac_10    141.469574
hvac_11      1.346326
hvac_12      1.813000
Name: 1185, dtype: float64


In [35]:
out_overall[2014]['Austin'].ix[26]['aggregate_6']

1299.5018310546875

In [36]:
[2014]['Austin'][26][7] -> all to Nan
[2014]['Austin'][187][7] -> all to Nan
[2014]['Austin'][457][3] -> all to Nan
[2014]['Austin'][457][4] -> all to Nan
[2014]['Austin'][457][5] -> all to Nan
[2014]['Austin'][457][6] -> all to Nan
[2014]['Austin'][457][7] -> all to Nan

[2014]['Austin'][580][6] -> all to Nan
[2014]['Austin'][772] ->delete all








TypeError: list indices must be integers, not str

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/nipunbatra/anaconda/lib/python2.7/lib-tk/Tkinter.py", line 1536, in __call__
    return self.func(*args)
  File "/Users/nipunbatra/anaconda/lib/python2.7/site-packages/matplotlib/backends/backend_tkagg.py", line 423, in button_release_event
    FigureCanvasBase.button_release_event(self, x, y, num, guiEvent=event)
  File "/Users/nipunbatra/anaconda/lib/python2.7/site-packages/matplotlib/backend_bases.py", line 1926, in button_release_event
    self.callbacks.process(s, event)
  File "/Users/nipunbatra/anaconda/lib/python2.7/site-packages/matplotlib/cbook.py", line 563, in process
    proxy(*args, **kwargs)
  File "/Users/nipunbatra/anaconda/lib/python2.7/site-packages/matplotlib/cbook.py", line 430, in __call__
    return mtd(*args, **kwargs)
  File "/Users/nipunbatra/anaconda/lib/python2.7/site-packages/matplotlib/widgets.py", line 229, in _release
    func(event)
  File "<ipython-input-31-f44945a9ccd7>", 

In [7]:
out_overall = {}
for year, year_dict in years.iteritems(): 
    out_overall[year] = {}
    to_ignore = []
    for city in cities[:]:
        print city, year
        city_data = metadata_df[metadata_df['city'] == city]
        city_homes = city_data.index.values.astype('int')
        out = {}
        for home in city_homes[:]:
            
            if home in to_ignore:
                continue
            try:
                data_df = st['/%d' %home][year_dict['start']:year_dict['end']]
                data_df_resampled = data_df.resample("1M", how="sum")
                df_res_kwh = data_df_resampled.div(1000)
                df_res_kwh.index = df_res_kwh.index.month
                # Find months where we have less than 28 days worth data and ignore those months
                day_res = data_df.resample("1D", how="sum").dropna()
                a = pd.value_counts(day_res.index.month)
                lt_28_days = a[a<28].index.values
                
                for m in lt_28_days:
                    df_res_kwh.loc[m] = np.NAN
                temp = {}
                for feed_name, feed_value in feeds.iteritems():
                    temp[feed_value] = []
                    for month in range(year_dict['start_month'], year_dict['end_month']+1):


                        try:
                            temp[feed_value].append(df_res_kwh.loc[month, feed_name])
                        except:
                            temp[feed_value].append(np.NAN)
                out[home] = np.hstack(temp.values())




            except Exception, e:
                pass
        appliances = temp.keys()
        col_headers = []
        for appliance in appliances:
            for month in range(1, 13):
                col_headers.append(appliance+"_"+str(month))
        df = pd.DataFrame(out).T
        df.columns = col_headers

        # Read metadata
        mdf = pd.read_csv("metadata/%s_md.csv" %city.replace(" ",""), index_col=0)
        for column in mdf.columns:
            df[column] = mdf[column]

        city = city.replace(" ","")
        out_overall[year][city] =df

        tdf = df[['aggregate_'+str(month) for month in range(1, 13)]].dropna().copy()
        full_agg = {}
     

        for home in df.index:
            if home in tdf.index:
                full_agg[home] = 1
            

            else:
                full_agg[home] = 0
     

Austin 2016


/Users/nipunbatra/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:16: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  app.launch_new_instance()
/Users/nipunbatra/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:20: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()


Houston 2016
San Diego 2016
Boulder 2016
Austin 2014
Houston 2014
San Diego 2014
Boulder 2014
Austin 2015
Houston 2015
San Diego 2015
Boulder 2015


In [9]:
out_overall[2014]['Austin'].ix[4590]

hvac_1                0.086317
hvac_2               20.471533
hvac_3                7.657416
hvac_4              196.112366
hvac_5              383.484344
hvac_6              814.079346
hvac_7              832.334961
hvac_8             1264.597290
hvac_9                     NaN
hvac_10                    NaN
hvac_11                    NaN
hvac_12                    NaN
wm_1                  3.894867
wm_2                  4.088883
wm_3                  5.350867
wm_4                  4.677467
wm_5                  2.040100
wm_6                  3.212350
wm_7                  5.303050
wm_8                  5.241133
wm_9                       NaN
wm_10                 5.608666
wm_11                 7.121400
wm_12                 8.528350
fridge_1             39.481869
fridge_2             38.179535
fridge_3             43.660366
fridge_4             44.056084
fridge_5             52.707150
fridge_6             53.328602
                      ...     
dw_3                  8.006283
dw_4    

In [10]:
def remove_lt(out_overall):
    for region in ['SanDiego','Austin']:
        for year in [2014, 2015, 2016]:
            df = out_overall[year][region]
            to_remove_agg_lt_appliance = []
            for appliance in ['hvac','wm','dw','mw','fridge','oven']:
                for month in range(1, 13):
                    a=df.query('aggregate_%d<1.35*%s_%d' %(month, appliance, month))
                    if len(a):
                        for x in a.index.values:
                            if x not in to_remove_agg_lt_appliance:
                                to_remove_agg_lt_appliance.append(x)
            print to_remove_agg_lt_appliance, year, region
            out_overall[year][region] = out_overall[year][region].drop(to_remove_agg_lt_appliance)
            
    return out_overall

In [11]:
out_overall = remove_lt(out_overall)

[] 2014 SanDiego
[] 2015 SanDiego
[] 2016 SanDiego
[] 2014 Austin
[] 2015 Austin
[] 2016 Austin


### Removing outliers based on aggregate

1. Remove homes where aggregate less than 150 units
2. Remove high energy using z-score test

In [78]:
def is_outlier(points, thresh=3.5):
    """
    Returns a boolean array with True if points are outliers and False 
    otherwise.

    Parameters:
    -----------
        points : An numobservations by numdimensions array of observations
        thresh : The modified z-score to use as a threshold. Observations with
            a modified z-score (based on the median absolute deviation) greater
            than this value will be classified as outliers.

    Returns:
    --------
        mask : A numobservations-length boolean array.

    References:
    ----------
        Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
        Handle Outliers", The ASQC Basic References in Quality Control:
        Statistical Techniques, Edward F. Mykytka, Ph.D., Editor. 
    """
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

In [79]:
for region in ['Austin']:
    for year in [2014, 2015, 2016]:
        to_remove = []
        for month in range(1, 13):
            df = out_overall[year][region]
            # Dropping outlier
            d = df['aggregate_%d' %month].dropna()
            drop_outlier = d.index[is_outlier(d)].values
            for x in drop_outlier:
                if x not in to_remove:
                    to_remove.append(x)
            
            # Drop lt
            drop_lt = d[d<150].index
            for x in drop_lt:
                if x not in to_remove:
                    to_remove.append(x)
            
        print region, year, to_remove
        out_overall[year][region] =  out_overall[year][region].drop(to_remove)
            
            
            

Austin 2014 [59, 222, 1507, 1700, 2995, 4154, 5357, 5673, 8079, 8197, 9356, 9647, 9745, 9771, 9922, 1069, 9578, 4957, 6799, 739, 3519, 5552, 7013, 7901, 7016, 9484, 1854, 8589, 9931, 252, 3631, 7512, 9701, 1500, 8031, 3299, 5456, 491, 9935, 2129, 9248, 3268, 3789, 3500, 6691, 7510, 2458, 4000, 7638, 2510, 8669]
Austin 2015 [59, 77, 222, 1507, 1700, 1994, 3916, 4154, 5673, 6691, 6799, 7016, 7510, 7788, 7965, 8197, 8419, 8669, 9356, 9647, 9745, 9771, 9922, 9935, 9938, 7512, 6941, 739, 3299, 3510, 7638, 9578, 9803, 3126, 3401, 4957, 5786, 898, 7900, 9248, 645, 3268, 4674, 5817, 6324, 4505, 7764, 4296, 252, 5784, 936]
Austin 2016 [222, 1507, 1700, 3126, 3401, 5357, 6691, 6799, 7510, 8197, 9356, 9647, 9922, 9935, 9938, 252, 4375, 9701, 7016, 3893, 8092, 2018, 4957, 1310, 3268, 8467, 9631, 7719, 2638, 4590, 3916, 8730]


In [80]:
%matplotlib inline

In [82]:
import pickle

In [83]:
pickle.dump(out_overall, open('metadata/all_regions_years_austin_cleaned.pkl','w'))